# Model Selection Via Transfer Learning

In [1]:
import os
import warnings
from datetime import datetime

import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from keras import applications
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical
import keras.backend as K

warnings.filterwarnings('ignore')

2024-03-21 06:51:43.495837: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 06:51:43.495959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 06:51:43.651566: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_data_dir = '/kaggle/input/landmark-identification/FinalData/Training_Data'
validation_data_dir = '/kaggle/input/landmark-identification/FinalData/Validation_Data'
test_data_dir= '/kaggle/input/landmark-identification/FinalData/Test_Data'

train_labels_file = '/kaggle/input/landmark-identification/Labels/training_labels.npy'
validation_labels_file = '/kaggle/input/landmark-identification/Labels/validation_labels.npy'
test_labels_file = '/kaggle/input/landmark-identification/Labels/test_labels.npy'

img_width, img_height = 224, 224
num_classes = 10
epochs = 16
batch_size = 512

In [3]:
train_labels = np.load(open(train_labels_file, 'rb'))
validation_labels = np.load(open(validation_labels_file, 'rb'))
test_labels = np.load(open(test_labels_file, 'rb'))

## Converting images to feature vectors using weights from ImageNet

In [4]:
def images_to_feature_vectors(model, directory, batch_size, steps):
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator()
    generator = datagen.flow_from_directory(
        directory,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        class_mode = None,
        shuffle = False) # Keep the data in the same order
    
    features = model.predict(generator, steps, verbose=1) 
    
    return features

## Train the top part of the model

In [5]:
def top_model(training_features, validation_features, batch_size, epochs):
    model = Sequential()
    model.add(Flatten(input_shape=training_features.shape[1:], name='Main_input'))
    model.add(BatchNormalization())
    model.add(Dense(1096, activation='relu', name='D1'))
    model.add(Dense(512, activation='relu', name='D2'))
    model.add(Dense(num_classes, activation='softmax', name='Main_output'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])
    train_labels_onehot = to_categorical(train_labels, num_classes)            #One Hot Encoder
    validation_labels_onehot = to_categorical(validation_labels, num_classes)  #One Hot Encoder
    
    model.fit(training_features, train_labels_onehot,
              epochs = epochs,
              batch_size = batch_size,
              validation_data = (validation_features, validation_labels_onehot))
   
    return model

## VGG-16 Training and Evaluation

In [6]:
start_time = datetime.now()

# Batch size has to be a multiple of the number of images  to keep our vectors consistents
training_batch_size = 1 # batch size for feature pre-training
validation_batch_size = 1 # batch size for feature pre-training

model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3)) #VGG16 trained on imagenet
training_features = images_to_feature_vectors(model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

end_time = datetime.now()
features_extraction_time = end_time - start_time
print('Features extraction time : {}'.format(features_extraction_time))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 61 images belonging to 10 classes.
24/61 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step

I0000 00:00:1711003921.141493     135 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step
Found 10 images belonging to 5 classes.
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Features extraction time : 0:00:07.198492


In [7]:
start_time = datetime.now()

model = top_model(training_features, validation_features, batch_size, epochs)

end_time = datetime.now()
training_time = end_time - start_time
print('Total training duration : {}'.format(training_time))

Epoch 1/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.0492 - loss: 3.2050

W0000 00:00:1711003924.672288     136 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0492 - loss: 3.2050 - val_accuracy: 0.8000 - val_loss: 4.4700
Epoch 2/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 0.9000 - val_loss: 7.5215
Epoch 3/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 8.3255e-04 - val_accuracy: 0.9000 - val_loss: 10.3270
Epoch 4/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.9836 - loss: 0.0398 - val_accuracy: 0.9000 - val_loss: 12.6792
Epoch 5/16


W0000 00:00:1711003925.360431     134 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 1.0000 - loss: 8.7085e-04 - val_accuracy: 0.8000 - val_loss: 14.6389
Epoch 6/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 6.5188e-04 - val_accuracy: 0.8000 - val_loss: 16.6623
Epoch 7/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 3.5215e-04 - val_accuracy: 0.8000 - val_loss: 18.4625
Epoch 8/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9836 - loss: 0.0426 - val_accuracy: 0.9000 - val_loss: 17.7691
Epoch 9/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 1.0000 - loss: 5.5030e-06 - val_accuracy: 0.9000 - val_loss: 17.7410
Epoch 10/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 1.0000 - loss: 2.3548e-06 - val_accuracy: 0.9000 - val_loss: 17.6656
Epoch 11/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 1.3425e-06 - val_accuracy: 0.9000 - val_loss: 17.5129
Epoch 12/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 1.0000 - loss: 7.7778e-07 - val_accuracy:

In [8]:
train_labels_onehot = to_categorical(train_labels, num_classes)
loss, training_accuracy = model.evaluate(training_features,train_labels_onehot)

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 550ms/step - accuracy: 1.0000 - loss: 3.7253e-08

W0000 00:00:1711003926.903305     135 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 541ms/step - accuracy: 1.0000 - loss: 2.4835e-08


W0000 00:00:1711003927.440311     137 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [9]:
validation_labels_onehot = to_categorical(validation_labels, num_classes)
loss, validation_accuracy = model.evaluate(validation_features,validation_labels_onehot)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9000 - loss: 16.2580


In [10]:
'Training Accuracy : '+ str(round(training_accuracy*100,1)) + '%'

'Training Accuracy : 100.0%'

In [11]:
'Validation Accuracy : '+ str(round(validation_accuracy*100,1)) + '%'

'Validation Accuracy : 90.0%'

In [12]:
vgg16_results = (round(training_accuracy*100,1), round(validation_accuracy*100,1), round(features_extraction_time.total_seconds()), round(training_time.total_seconds()))

## ResNet Training and Evaluation

In [13]:
start_time = datetime.now()

model = applications.ResNet50(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3)) #VGG16 trained on imagenet
training_features = images_to_feature_vectors(model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

end_time = datetime.now()
features_extraction_time = end_time - start_time
print('Features extraction time : {}'.format(features_extraction_time))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Found 61 images belonging to 10 classes.
61/61 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step
Found 10 images belonging to 5 classes.
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Features extraction time : 0:00:08.158400


In [14]:
start_time = datetime.now()

model = top_model(training_features, validation_features, batch_size, epochs)

end_time = datetime.now()
training_time = end_time - start_time
print('Total training duration : {}'.format(training_time))

Epoch 1/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.0656 - loss: 2.8471

W0000 00:00:1711003939.055252     135 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0656 - loss: 2.8471 - val_accuracy: 0.8000 - val_loss: 5.4461
Epoch 2/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.8000 - val_loss: 7.2854
Epoch 3/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 1.0000 - loss: 0.0000e+00

W0000 00:00:1711003939.867798     134 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.8000 - val_loss: 9.0144
Epoch 4/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 0.9000 - val_loss: 11.0881
Epoch 5/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.9672 - loss: 0.4842 - val_accuracy: 0.9000 - val_loss: 12.1607
Epoch 6/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.9836 - loss: 0.1031 - val_accuracy: 0.9000 - val_loss: 12.9323
Epoch 7/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.9000 - val_loss: 13.6592
Epoch 8/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.9000 - val_loss: 14.2669
Epoch 9/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.9000 - val_loss: 14.7861
Epoch 10/16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.9

In [15]:
loss, training_accuracy = model.evaluate(training_features,train_labels_onehot)

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 762ms/step - accuracy: 0.9375 - loss: 3.5973

W0000 00:00:1711003942.330358     137 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 677ms/step - accuracy: 0.9573 - loss: 2.3982


W0000 00:00:1711003943.003240     137 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [16]:
loss, validation_accuracy = model.evaluate(validation_features,validation_labels_onehot)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8000 - loss: 21.8373


In [17]:
'Training Accuracy : '+ str(round(training_accuracy*100,1)) + '%'

'Training Accuracy : 96.7%'

In [18]:
'Validation Accuracy : '+ str(round(validation_accuracy*100,1)) + '%'

'Validation Accuracy : 80.0%'

In [19]:
resnet_results = (round(training_accuracy*100,1), round(validation_accuracy*100,1), round(features_extraction_time.total_seconds()), round(training_time.total_seconds()))

In [20]:
results = {'Model': ['VGG16', 'Resnet'], 
           'Training Accuracy(%)': [vgg16_results[0], resnet_results[0]],
           'Validation Accuracy(%)': [vgg16_results[1], resnet_results[1]],
           'Features Extraction time(s)': [vgg16_results[2], resnet_results[2]],
           'Training time(s)': [vgg16_results[3], resnet_results[3]]}
df = pd.DataFrame(data=results)

In [21]:
df

,Model,Training Accuracy(%),Validation Accuracy(%),Features Extraction time(s),Training time(s)
0,VGG16,100.0,90.0,7,4
1,Resnet,96.7,80.0,8,6
